This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# The mathematical building blocks of neural networks

## A first look at a neural network

**Loading the MNIST dataset in Keras**

In [2]:
#!pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=5a8b23731a8f1888ec84509f89107120ba72a675a165acb821840063f947608d
  Stored in directory: c:\users\jonat\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.0
    Uninstalling importlib-metadata-3.10.0:
      Successfully uninstalled importlib-metadata-3.10.0


In [3]:
from tensorflow.keras.datasets import mnist

#carga de datos
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 2s 0us/step


In [4]:
train_images.shape #forma del data set
#primera entrada, número de registros, y despues cada registro es una matriz de 2x2

(60000, 28, 28)

In [5]:
len(train_labels) #corroborando el largo de la data

60000

In [6]:
train_labels #checando las etiquetas

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [7]:
test_images.shape #checando la forma del test

(10000, 28, 28)

In [8]:
len(test_labels)

10000

In [9]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

**The network architecture**

Arquitectura de una red neuronal

    - recordemos que en estas redes la capa de entrada se infiere: es decir en automatico, la red identifica de cuantos nodos es la entarda, en este caso como cada registro es una matriz de 28x28, la primera capa de la red es de 28x28 = 784, es decir ahora tenemos un dataframe de 60k registros y 784 columnas (caracteristicas)

In [10]:
from tensorflow import keras
from tensorflow.keras import layers

#modelo secuencial de red neuronal

model = keras.Sequential([
    # segunda capa con 512 neuronas, con funcion de activación relu
    layers.Dense(512, activation = "relu"),
    # capa de salida, claramete es 10 porque buscamos clasificar 10 numeros
    # la salida de la capa es un array de 10 entradas donde cada entrada es 
    # una probabilidad de ser una etiqueta, se elige la de la etiqueta más alta
    layers.Dense(10, activation = "softmax") #activación softmax porque es para clasificación multiple
])

**The compilation step**

    - La compilación del modelo es asignar la funcion de optimizacion, y la funcion de perdida sobre la que se optimiza y también pedimos la metrica de rendimiento del algoritmo

In [11]:
model.compile(optimizer = "rmsprop", #funcion de optimizacion
              loss = "sparse_categorical_crossentropy", #funcion de perdida para categoricos
              metrics = ["accuracy"])

**Preparing the image data**

In [13]:
#"desdoblamos" la imagen
train_images = train_images.reshape((60000, 28 * 28))

# optimizamos el tamaño de los datos y se normalizan los valores de los pixeles
# es decir, en cada matriz de 28x28 se toman valores de 0-255, y como es más facil trabajar
# con numeros pequeños, esa es la razón por la cual se divide entre 255
train_images = train_images.astype("float32") / 255

#se hace lo mismo para el conjunto de prueba
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

**"Fitting" the model**

    - para alimentar el modelo, se le ingesta el vector de caracteristicas train, la etiqueta label, el numero de veces a iterar "epochs y el batch size, lo cual es un muestreo de entrenamiento hasta acabarse los datos, es decir, se entrena de 128 en 128 hasta acabar el largo del dataset 60k registros

In [14]:
model.fit(train_images, train_labels, epochs = 5, batch_size = 128)

Epoch 1/5
469/469 [==============================] - 5s 9ms/step - loss: 1.4774 - accuracy: 0.6754
Epoch 2/5
469/469 [==============================] - 4s 9ms/step - loss: 0.5939 - accuracy: 0.8516
Epoch 3/5
469/469 [==============================] - 4s 9ms/step - loss: 0.4155 - accuracy: 0.8876
Epoch 4/5
469/469 [==============================] - 4s 9ms/step - loss: 0.3562 - accuracy: 0.9003
Epoch 5/5
469/469 [==============================] - 4s 9ms/step - loss: 0.3264 - accuracy: 0.9069


la perdida disminuye mientras el accuracy crece

**Using the model to make predictions**

    - hacemos la prediccion sobre los datos que no se han visto (conjunto de test)

In [15]:
test_digits = test_images[0:10]
predictions = model.predict(test_digits)
predictions[0] # vector de probas 

array([5.2466254e-05, 4.3620105e-08, 1.5195715e-05, 3.9135251e-04,
       1.3093598e-06, 2.7699052e-05, 1.0298190e-08, 9.9806648e-01,
       5.7623429e-06, 1.4396828e-03], dtype=float32)

In [16]:
predictions[0].argmax() #nos predice que es un 7

7

In [18]:
predictions[0][7] #pseudo-proba de que que sea un 7

0.9980665

In [20]:
test_labels[0] #corroboramos la etiqueta

7

**Evaluating the model on new data**
    - evalua el modelo,es decir qué metricas (funcion de perdida y accuracy) estoy obteniendo con el modelo entrenado pero ahora aplicando lo obtenido al conjunto de test

In [21]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"test_acc: {test_acc}")

313/313 [==============================] - 1s 4ms/step - loss: 0.3055 - accuracy: 0.9113
test_acc: 0.911300003528595


## Data representations for neural networks

### Scalars (rank-0 tensors)

In [22]:
import numpy as np
x = np.array(12)
x

array(12)

In [23]:
x.ndim

0

### Vectors (rank-1 tensors)

In [24]:
x = np.array([12, 3, 6, 14, 7])
x

array([12,  3,  6, 14,  7])

In [25]:
x.ndim

1

### Matrices (rank-2 tensors)

In [26]:
x = np.array([[5, 78, 2, 34, 0],
              [6, 79, 3, 35, 1],
              [7, 80, 4, 36, 2]])
x.ndim

2

### Rank-3 and higher-rank tensors

In [27]:
x = np.array([[[5, 78, 2, 34, 0],
               [6, 79, 3, 35, 1],
               [7, 80, 4, 36, 2]],
              [[5, 78, 2, 34, 0],
               [6, 79, 3, 35, 1],
               [7, 80, 4, 36, 2]],
              [[5, 78, 2, 34, 0],
               [6, 79, 3, 35, 1],
               [7, 80, 4, 36, 2]]])
x.ndim

3